In [1]:
import numpy as np
from matplotlib import pyplot
from scipy import linalg
import copy
import iMPS
from scipy.sparse import linalg
import funcs

In [2]:
eigs=[]
lam1s=[]
lam2s=[]
lam3s=[]
lam4s=[]
dims=[]
X = np.array([[0,1],[1,0]])
Z = np.array([[1,0],[0,-1]])
for h in np.arange(0.40,0.41,0.01):   
    
    h1 = h
    h2 =0
    
    B = np.zeros([1,1,4])
    B[0,0] =(linalg.expm(h*Z)).reshape([4,])
    MPS = iMPS.iMPS()
    MPS.construct_from_tensor_list([B,B])
    MPS.svd_threshold =1e-10
    trans = funcs.single_trans(h1=h1,h2=h2)
    
    MPO = iMPS.iMPO()
    MPO.construct_from_tensor_list([trans,trans])
    MPS_power = iMPS.MPS_power_method(MPS,MPO,max_bond=20)
    MPS_power.update(500)
    

    
    MPS_two_site = iMPS.MPS_power_method_twosite_update2(MPS,MPO,20)
    
    MPS_two_site.svd_threshold=1e-10
    MPS_two_site.MPS2r = copy.deepcopy(MPS)
    MPS_two_site.init_MPS2()
    
    
    MPS_two_site.init_env()
    MPS_two_site.update_MPS2(loop=500)
    for _ in range(0):
        
        MPS1r = iMPS.iMPS()
        MPS1r.construct_from_tensor_list([MPS_two_site.MPS2r.B[0],MPS_two_site.MPS2r.B[1]])
        
        MPS1l = iMPS.iMPS()
        MPS1l.construct_from_tensor_list([MPS_two_site.MPS2l.B[0],MPS_two_site.MPS2l.B[1]])
        
        
        MPS_two_site = iMPS.MPS_power_method_twosite_update2(MPS1r,MPO,20)
        MPS_two_site.MPS1l = copy.deepcopy(MPS1r)
        MPS_two_site.svd_threshold=1e-10
        MPS_two_site.MPS2l = copy.deepcopy(MPS1l)
        MPS_two_site.MPS2r = copy.deepcopy(MPS1r)
        #MPS_two_site.init_MPS2()
        
        
        MPS_two_site.init_env()
        MPS_two_site.update_MPS2(loop=100)
    
    """
    
    MPS_two_site = iMPS.MPS_power_method_twosite_update2(MPS8,MPO,20)
    
    MPS_two_site.svd_threshold=1e-10
    MPS_two_site.MPS2r = copy.deepcopy(MPS8)
    MPS_two_site.init_MPS2()
    
    
    MPS_two_site.init_env()
    MPS_two_site.update_MPS2(loop=400)"""
    
    """T3 = np.zeros([1,1,4,4])
    T3[0,0,:,:] = np.kron(X,X)
    MPO3 = iMPS.iMPO()
    MPO3.construct_from_tensor_list([T3])
    strap4 = iMPS.strap(MPS,MPO3,MPS)
    lam4 = strap4.calculate_eig()
    lam4s.append(lam4)"""
    
    
    T3 = np.zeros([1,1,4,4])
    T3[0,0,:,:] = np.kron(X,X)
    MPO4 = iMPS.iMPO()
    MPO4.construct_from_tensor_list([T3,T3])
    strap4 = iMPS.strap(MPS,MPO4,MPS)
    lam4 = strap4.calculate_eig()
    lam4s.append(lam4)
    
    
    B1 = MPS_two_site.MPS2l.B[0]
    s0 = np.diag(MPS_two_site.MPS2l.s[0])
    s1 = np.diag(MPS_two_site.MPS2l.s[1])
    B1 = funcs.row_contract23(np.linalg.inv(s0),B1)
    B1= funcs.row_contract32(B1,s1)
    
    MPS8 = iMPS.iMPS()
    MPS8.construct_from_tensor_list([MPS_two_site.MPS2r.B[0],MPS_two_site.MPS2r.B[1]])
    
    strap4 = iMPS.strap(MPS8,MPO4,MPS8)
    lam4 = strap4.calculate_eig()
    lam1s.append(lam4)
    
    MPS9 = iMPS.iMPS()
    MPS9.construct_from_tensor_list([MPS_two_site.MPS2l.B[0],MPS_two_site.MPS2l.B[1]])
    
    
    strap4 = iMPS.strap(MPS8,None,MPS8)
    lam4 = strap4.calculate_eig()
    lam2s.append(lam4)
    
    dims.append(MPS_two_site.MPS2r.chi[1])
    
    strap4 = iMPS.strap(MPS8,None,MPS9)
    lam4 = strap4.calculate_eig()
    lam3s.append(lam4)
    
    
    
    
   
lam4s = np.abs(lam4s)
lam1s = np.abs(lam1s)
lam2s = np.abs(lam2s)
lam3s = np.abs(lam3s)

In [3]:
print(MPS_two_site.MPS2l.B[0].shape)
print(MPS_two_site.MPS2l.B[1].shape)
print(MPS_two_site.MPS2r.B[0].shape)
print(MPS_two_site.MPS2r.B[1].shape)
print(MPS_two_site.MPS2l.s[1].shape)
print(MPS_two_site.MPS2r.s[1].shape)

print(MPS_two_site.MPS1l.B[0].shape)
print(MPS_two_site.MPS1l.B[1].shape)
print(MPS_two_site.MPS1r.B[0].shape)
print(MPS_two_site.MPS1r.B[1].shape)
print(MPS_two_site.MPS1l.s[1].shape)
print(MPS_two_site.MPS1r.s[1].shape)

print(MPS_two_site.transfer_matrix(0).shape)
print(MPS_two_site.transfer_matrix(1).shape)


print(MPS_two_site.El[0].shape)
print(MPS_two_site.El[1].shape)



print(MPS_two_site.Er[0].shape)
print(MPS_two_site.Er[1].shape)

(4, 8, 4)
(4, 4, 4)
(4, 4, 4)
(8, 4, 4)
(8,)
(8,)
(4, 4, 4)
(4, 4, 4)
(4, 4, 4)
(4, 4, 4)
(4,)
(4,)
(64, 64)
(64, 64)
(64,)
(128,)
(64,)
(128,)


In [3]:
print(np.abs(lam1s))
print(lam4s)

print(np.abs(lam2s))

print(np.abs(lam3s))

[0.83543656]
[0.99999882]
[1.]
[0.96292685]


In [52]:
MPS_two_site = iMPS.MPS_power_method_twosite_update2(MPS,MPO,20)


MPS_two_site.MPS2r = copy.deepcopy(MPS)
MPS_two_site.svd_threshold = 1e-6
MPS_two_site.init_MPS2()
MPS_two_site.init_env()

for _ in range(30):
    s0 =np.diag(MPS_two_site.MPS1r.s[0])
    s1 =np.diag(MPS_two_site.MPS1r.s[1])

    T1 = funcs.row_contract33(funcs.row_contract32(MPS_two_site.MPS1l.B[0],s1),MPS_two_site.MPS1r.B[1])
    T2 = funcs.row_contract33(MPS_two_site.MPS1r.B[0],MPS_two_site.MPS1r.B[1])

    #print(np.linalg.norm(T1-T2))
    #print(MPS_two_site.Er[0]/MPS_two_site.Er[0][0])
    """T3 = MPS_two_site.new_tensor(0).conj()

    print(np.linalg.norm(T1-T3/T3[0,0,0,0]*T1[0,0,0,0]))
    """
    MPS8 = iMPS.iMPS()
    s0 = np.diag(MPS_two_site.MPS2l.s[0])
    s1 = np.diag(MPS_two_site.MPS2l.s[1])
    MPS8.construct_from_tensor_list([MPS_two_site.MPS2r.B[0],MPS_two_site.MPS2r.B[1]])
    strap4 = iMPS.strap(MPS,None,MPS8)
    lam4 = strap4.calculate_eig()
    print(abs(lam4))
    
    MPS_two_site.update_MPS2(loop=30)

"""#B1_new,lam,B2_new = MPS_two_site.two_site_svd(0)
new_M = T3
new_M = np.transpose(new_M,[0,2,1,3])
w = np.shape(new_M)
            
new_M = new_M.reshape([w[0]*w[1],w[2]*w[3]])
            
U,lam,V = np.linalg.svd(new_M)
dim = np.sum(lam>1e-10)

U = U[:,:dim]
lam = lam[:dim]
V = V[:dim,:]
lam = lam/np.linalg.norm(lam)
            
B_new_left = np.reshape(U@np.diag(lam),[-1,4,dim])
B_new_left  = np.transpose(B_new_left ,[0,2,1])
B_new_left  = funcs.row_contract23(np.linalg.inv(s0),B_new_left)
        
B_new_right = np.reshape(V,[dim,-1,4])





T4 = funcs.row_contract33(B_new_left,B_new_right)
print(np.linalg.norm(T4*T2[0,0,0,0]/T4[0,0,0,0]-T2))
MPS1 = iMPS.iMPS()
MPS1.construct_from_tensor_list([B_new_left,B_new_right])
strap4 = iMPS.strap(MPS1,MPO4,MPS1)
lam4 = strap4.calculate_eig()
print(abs(lam4))"""

1.0000000000000002
0.8871249155224421
0.9378530835736146
0.8579569631017758


ValueError: cannot multiply <49x36 _CustomLinearOperator with dtype=complex128> and <49x49 _CustomLinearOperator with dtype=complex128>: shape mismatch

In [43]:
MPS_two_site.MPS2l.s[1].shape

(4,)

In [42]:
MPS_two_site.MPS2l.B[0].shape

(4, 4, 4)

In [ ]:
MPS_two_site.MPS1l.B[0][:,:3,:]/MPS_two_site.MPS1l.B[0][0,0,0]

In [ ]:
B_new_left[:,:3,:]/B_new_left[0,0,0]

In [ ]:
MPS_two_site.MPS1l.B[0][:,:4,:]/MPS_two_site.MPS1l.B[0][0,0,0]-B_new_left[:,:4,:]/B_new_left[0,0,0]

In [15]:
MPS1 = iMPS.iMPS()
MPS1.construct_from_tensor_list([B_new_right[:4,:,:]])

MPS2 = iMPS.iMPS()
MPS2.construct_from_tensor_list([MPS_two_site.MPS1r.B[0][:4,:,:]])

strap = iMPS.strap(MPS1,MPO = None,MPS2=MPS1)
print(abs(strap.calculate_eig()))
strap = iMPS.strap(MPS2,MPO = None,MPS2=MPS2)
print(abs(strap.calculate_eig()))
strap = iMPS.strap(MPS1,MPO = None,MPS2=MPS2)
print(abs(strap.calculate_eig()))

1.0000000000000007
1.0000000000000018
0.9914887292385939


In [14]:
np.linalg.eig((MPS1.B[0]/MPS1.B[0][0,0,0]-MPS2.B[0]/MPS2.B[0][0,0,0])[:,:,3])

(array([-0.93271333+0.56968439j,  0.01278519+0.06610382j,
        -0.14762512+0.16472698j, -0.12835112+0.25992883j]),
 array([[ 1.72603339e-03+0.00239108j,  1.04616929e-02-0.03076372j,
         -3.38982884e-03+0.00124647j,  1.88549543e-04+0.00132374j],
        [ 3.14602086e-01+0.07521469j,  4.05671693e-02-0.01896337j,
         -1.55039410e-03-0.00421524j,  2.62157777e-03-0.00260507j],
        [-4.91407596e-01-0.14923381j,  3.93080640e-01+0.02250958j,
          5.33147333e-02-0.13343362j, -5.19270603e-02-0.01798423j],
        [ 7.94737332e-01+0.j        ,  9.17561862e-01+0.j        ,
          9.89605877e-01+0.j        ,  9.98481197e-01+0.j        ]]))

In [13]:
np.linalg.eig((MPS1.B[0]/MPS1.B[0][0,0,0]-MPS2.B[0]/MPS2.B[0][0,0,0])[:,:,3])

(array([-0.93070313+0.5679463j ,  0.01268056+0.06598344j,
        -0.14741288+0.1652505j , -0.1280139 +0.25979722j]),
 array([[ 1.71544601e-03+0.00241043j,  1.08305677e-02-0.03078749j,
         -3.39698670e-03+0.00122712j,  1.59221333e-04+0.00133609j],
        [-3.14994276e-01-0.07591464j, -4.09747019e-02+0.01872691j,
          1.55305998e-03+0.00426109j, -2.58764284e-03+0.00260578j],
        [ 4.91526884e-01+0.15223269j, -3.93002264e-01-0.02736462j,
         -5.40585372e-02+0.13359993j,  5.19880760e-02+0.01948589j],
        [ 7.93872377e-01+0.j        ,  9.17445151e-01+0.j        ,
          9.89542884e-01+0.j        ,  9.98449920e-01+0.j        ]]))

In [55]:
np.random.seed(1)
A = np.random.random([4,4])+ np.random.random([4,4])*1j
A

array([[4.17022005e-01+0.4173048j , 7.20324493e-01+0.55868983j,
        1.14374817e-04+0.14038694j, 3.02332573e-01+0.19810149j],
       [1.46755891e-01+0.80074457j, 9.23385948e-02+0.96826158j,
        1.86260211e-01+0.31342418j, 3.45560727e-01+0.69232262j],
       [3.96767474e-01+0.87638915j, 5.38816734e-01+0.89460666j,
        4.19194514e-01+0.08504421j, 6.85219500e-01+0.03905478j],
       [2.04452250e-01+0.16983042j, 8.78117436e-01+0.8781425j ,
        2.73875932e-02+0.09834683j, 6.70467510e-01+0.42110763j]])

In [56]:
A = A+A.transpose().conj()
A = A*0.5j
print(A)

[[ 0.        +0.417022j    0.12102737+0.43354019j  0.36800111+0.19844092j
  -0.01413553+0.25339241j]
 [-0.12102737+0.43354019j  0.        +0.09233859j  0.29059124+0.36253847j
   0.09290994+0.61183908j]
 [-0.36800111+0.19844092j -0.29059124+0.36253847j  0.        +0.41919451j
   0.02964603+0.35630355j]
 [ 0.01413553+0.25339241j -0.09290994+0.61183908j -0.02964603+0.35630355j
   0.        +0.67046751j]]


In [57]:
theta=funcs.find_phase(A)

In [58]:
A/theta

array([[0.5       +0.j        , 0.51980494-0.14510909j,
        0.2379262 -0.44122505j, 0.3038118 +0.01694819j],
       [0.51980494+0.14510909j, 0.1107119 +0.j        ,
        0.43467547-0.34841236j, 0.73358129-0.11139693j],
       [0.2379262 +0.44122505j, 0.43467547+0.34841236j,
        0.50260479+0.j        , 0.42719994-0.03554492j],
       [0.3038118 -0.01694819j, 0.73358129+0.11139693j,
        0.42719994+0.03554492j, 0.80387546+0.j        ]])